In [1]:
import sys
import yfinance as yf
import requests
from bs4 import BeautifulSoup
from prophet import Prophet
from pandas.plotting import autocorrelation_plot
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta
import pandas_datareader as pdr
import pytictoc as tt
import plotly
from tickerTA import Ticker
from tickerTA import TechnicalAnalysis
from tradingStrategy1 import TradingStrategy1

# This is to avoid the SettingWithCopyWarning in pandas
pd.options.mode.copy_on_write = True 

/workspaces/AlgorithmicTrading/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_tickers500():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers500 = []
    for row in table.find_all('tr')[1:]:
        ticker = row.find('td').text.strip()
        tickers500.append(ticker)
    return tickers500

def add_z_score(data: pd.DataFrame, frame: str = 'Adj Close', window1 = 20) -> pd.DataFrame:
    data['Z Score ' + frame] = (data[frame] - data[frame].rolling(window= window1).mean()) / data[frame].rolling(window= window1).std()
    data.dropna(inplace=True)
    return data

def get_z_score(data: pd.DataFrame, frame: str = 'Adj Close', window1 = 20) -> pd.DataFrame:
    data['Z Score ' + frame] = (data[frame] - data[frame].rolling(window= window1).mean()) / data[frame].rolling(window= window1).std()
    data.dropna(inplace=True)
    return data['Z Score ' + frame]

def plot_total_cum_profit(self, start='2022-01-31', end='2024-01-31'):
    self['Total Cum Profit %'] = self['Profit %'].cumsum()
    self.reset_index(inplace=True)
    self.index = pd.to_datetime(self['Buy Date'])
    plt.figure(figsize=(20, 10))
    plt.plot(self['Buy Date'], self['Total Cum Profit %'], drawstyle="steps-post", label='Total Cumulative Profit', alpha=1)

    spy_df = yf.download('SPY', start, end, progress=False)
    spy_df['Control'] = spy_df['Close'] - spy_df['Open']
    spy_df['Control %'] = (spy_df['Control'] / spy_df['Open']) * 100
    spy_df['Control Cumulative %'] = spy_df['Control %'].cumsum()
    plt.plot(spy_df['Control Cumulative %'], drawstyle="steps-post", label='SPY', alpha=0.8)

    plt.title('Total Cumulative Profit % vs SPY')
    plt.legend(loc='upper left')
    plt.savefig('tradingStrategy1/total_cum_profit.png')
    # plt.show()

    plt.close()

In [3]:

if __name__ == '__main__':
    tickers500 = get_tickers500()
    stock_df = pd.DataFrame()
    tradingstrategy1_df = pd.DataFrame()

    for ticker in tickers500[0:2]:
        stock = Ticker(ticker, start='2022-01-31', end='2024-01-31')
        stock_df = stock_df._append(stock.df)
        techA = TechnicalAnalysis(stock)
        tradingS = TradingStrategy1(techA)
        tradingstrategy1_df = tradingstrategy1_df._append(tradingS.trades_df)

    plot_total_cum_profit(tradingstrategy1_df)
    print(tradingstrategy1_df['Total Cum Profit %'])



Buy Date
2022-04-19     4.30
2022-06-13     6.41
2022-08-29     2.29
2022-12-16   -10.33
2023-05-17     1.27
2023-09-26     0.71
2022-04-01    -3.38
2022-08-29   -10.03
2022-12-16    -3.29
2023-06-01    16.02
2023-08-21    16.59
Name: Total Cum Profit %, dtype: float64
